This notebook contains a workflo to adding contrast and correlation from GLCM texture features

References:
https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops

https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_glcm.html

https://prism.ucalgary.ca/bitstream/handle/1880/51900/texture%20tutorial%20v%203_0%20180206.pdf?sequence=11&isAllowed=y

https://stackoverflow.com/questions/50834170/image-texture-with-skimage

https://www.tandfonline.com/doi/epdf/10.1080/01431161.2016.1278314?needAccess=true&role=button

In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import sample_rasters as sr
from rasterio.crs import CRS
from rasterio.transform import rowcol

from shapely.geometry import box

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = pd.read_csv(sr.path_to_aoi_itemids_csv())

# csv with the points for which to add spectral window features
csv_name = 'spectral_window_model3070_train_2020.csv'
#fp = '/home/jovyan/msai4earth-esa/iceplant_detection/processing_results/model_3070/model3070_test_2020.csv'
fp = os.path.join(os.getcwd(),'temp',csv_name)
all_pts = pd.read_csv(fp)

window_r = 5  #(in meters?)
side = 2*window_r + 1

distances = [1]
angles = [0, np.pi/2]
props = ['contrast', 'correlation']

# ***************************************************
# ***************************************************

In [3]:
# temporary folder for aux rasters
folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
if os.path.exists(folp) == False:
    os.mkdir(folp)

In [4]:
t0 = time.time() # initial time tracker

sampled_pts = [] # sampled pts from each scene are collected here

N = len(itemids)  # counter to finish

crss = all_pts.pts_crs.unique()

for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    all_pts_scene = all_pts.loc[all_pts['naip_id'] == itemid]
    if len(all_pts_scene) !=0:
        for crs_str in crss:
            pts_scene = all_pts_scene[all_pts_scene.pts_crs == crs_str]  
    
            if len(pts_scene) !=0:
                crs = CRS.from_string(crs_str)
                pts_scene_df = sr.geodataframe_from_csv(df = pts_scene, lon_label='x', lat_label='y', crs=crs)
                pts_col = pts_scene_df.to_crs(naip_rast_r.rio.crs).geometry

                samples = []
                for pt in pts_col:
                    # make a small window out of raster, all bands
                    row, col = rowcol(naip_rast_r.rio.transform(), pt.x, pt.y)
                    windows = naip_rast_r[:, row-window_r:row+window_r+1, col-window_r:col+window_r+1]
                   
                    # calculate GLCM on that small window, all bands
                    # calculate the contrast and correlation on that window (upwards direction)
                    # add those as features for that pixel                    
                    pixel_feats = []
                    for angle in angles:
                        for band in range(4):
                            glcm = graycomatrix(windows[band],
                                        distances=distances,
                                        angles=angles)
                            pixel_feats.append(graycoprops(glcm, 'contrast')[0,0])
                            pixel_feats.append(graycoprops(glcm, 'correlation')[0,0])
                    samples.append(pixel_feats)
                    
                # ---------------------------------------
                # Add all derived spectral data to pts dataframe
                new_features = pd.DataFrame(np.vstack(samples), 
                                            columns = ['r_contE', 'r_corrE', 'g_contE', 'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE',
                                                       'r_contN', 'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN', 'nir_corrN'])   
                pts = pd.concat([pts_scene, new_features.set_index(pts_col.index)], axis=1)                

                # -----------------------------
                # collect all points from each polygon in the scene
                sampled_pts.append(pts)                    

    # ---------------------------------------
    # processing message
    N = N-1                
    print('REMAINING: ', N, 'scenes', end="\r")

print('FINISHED PROCESSING')       
     
# ---------------------------------------
# create data frame with all points
sampled_pts = pd.concat(sampled_pts).sort_index()                

FINISHED PROCESSINGss


In [5]:
sampled_pts

,x,y,pts_crs,aoi,naip_id,polygon_id,r,r_max,r_min,r_diff,...,nir_contE,nir_corrE,r_contN,r_corrN,g_contN,g_corrN,b_contN,b_corrN,nir_contN,nir_corrN
0,240948.401257,3.812109e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,53.0,82,85,77,8,...,47.209091,0.779393,71.918182,0.666608,23.890909,0.530555,19.700000,0.900987,47.209091,0.779393
1,238504.090827,3.810824e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,2.0,75,83,69,14,...,32.036364,0.896639,21.218182,0.979633,15.145455,0.981025,11.554545,0.990007,32.036364,0.896639
2,237541.520930,3.812742e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,44.0,68,75,67,8,...,25.145455,0.864418,53.954545,0.963070,31.100000,0.969574,27.618182,0.979680,25.145455,0.864418
3,238919.581523,3.811656e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,7.0,79,110,79,31,...,258.718182,0.594313,213.554545,0.905942,134.518182,0.911935,134.909091,0.925584,258.718182,0.594313
4,240990.693704,3.813885e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,48.0,82,87,79,8,...,4.327273,0.557339,9.245455,0.775899,3.490909,0.723760,1.009091,0.497996,4.327273,0.557339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,735332.842908,3.815994e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,28.0,92,98,82,16,...,148.390909,0.645066,267.300000,0.631255,207.300000,0.524841,112.290909,0.528251,148.390909,0.645066
1607,-120.484881,3.449711e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,NaN,97,118,89,29,...,44.881818,0.805088,74.272727,0.806723,46.136364,0.742219,22.136364,0.749503,44.881818,0.805088
1608,-120.485111,3.449345e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,NaN,109,110,89,21,...,30.545455,0.864088,56.772727,0.784410,28.072727,0.731816,18.536364,0.784205,30.545455,0.864088
1609,-120.468795,3.447044e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,NaN,107,128,93,35,...,24.618182,0.803617,60.381818,0.827843,45.309091,0.805710,38.236364,0.799639,24.618182,0.803617


In [6]:
sampled_pts.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'polygon_id', 'r', 'r_max',
       'r_min', 'r_diff', 'r_avg', 'r_entr', 'g', 'g_max', 'g_min', 'g_diff',
       'g_avg', 'g_entr', 'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr',
       'nir', 'nir_max', 'nir_min', 'nir_diff', 'nir_avg', 'nir_entr', 'ndvi',
       'ndvi_max', 'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr', 'year',
       'month', 'day_in_year', 'r_entr5', 'g_entr5', 'b_entr5', 'nir_entr5',
       'ndvi_entr5', 'iceplant', 'geometry', 'r_contE', 'r_corrE', 'g_contE',
       'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE', 'r_contN',
       'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN',
       'nir_corrN'],
      dtype='object')

In [7]:
sampled_pts = sampled_pts[['x', 'y', 'pts_crs', #  point location
             'aoi', 'naip_id', 'polygon_id',  # sampling info
             'r', 'r_max', 'r_min', 'r_diff', 'r_avg', 'r_entr', # spectral
             'g', 'g_max', 'g_min', 'g_diff', 'g_avg', 'g_entr',
             'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr',
             'nir', 'nir_max', 'nir_min', 'nir_diff', 'nir_avg', 'nir_entr',
             'ndvi', 'ndvi_max', 'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr',   
             'year', 'month', 'day_in_year', # date
             'r_contE', 'r_corrE', 'g_contE', 'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE',
             'r_contN', 'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN', 'nir_corrN',                           
            'r_entr5','g_entr5', 'b_entr5', 'nir_entr5', 'ndvi_entr5',                          
             'iceplant'
             ]] 


In [8]:
fp = os.path.join(os.getcwd(),'temp', 'glcm_'+csv_name)
sampled_pts.to_csv(fp, index=False)